# Spiel Definition
In diesem Kapitel werden Funktionen diskutiert, die nötig sind um das Spiel Mühle nach

$$G_{Nine Men's Morris} = 〈States,s0,Players,nextStates,finished,utility〉$$

zu definieren.

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game-utils.ipynb

## Spieler

Zunächst soll die Menge der Spieler definiert werden. Da Mühle mit zwei Spielern gespielt wird, die jeweils weiße oder schwarze Steine legen, werden die Spieler äquivalent benannt.
* Der beginnende Spieler weiß wird als `w` dargestellt,
* der gegnerische Spieler als `b`.

Für die Implementierung der Funktionen ist diese Menge nicht nötig. Aus Gründen der Vollständigkeit soll diese hier dennoch definiert werden.

In [ ]:
Player = {'w', 'b'}

## Zustände
Die Menge aller Zustände ist aufwendig zu berechnen und wird ebenfalls nicht für die Implementierung benötigt. da die Zustände berechnet werden, sobald diese benötigt werden. Deswegen wird hier auf eine Definition von $States$ verzichtet.

Der Startzustand $s_0$ hingegen wird zu Beginn des Spiel benötigt und ist im folgenden Definiert. Er bezeichnet ein leeres Spielbrett, bei dem beide Spieler noch alle ihre Steine legen müssen.
Ein Zustand (`state`) ist eine Tupel die
1. den Stapel (`stash`) von zu legenden Steinen und
2. das Spielbrett (`board`) selbst
beinhaltet.

Der Stapel ist eine Tupel $\langle w, b \rangle$, die die Anzahl der zu legenden Steine von Weiß $w$ und Schwarz $b$ beinhaltet. Diese werden in der genannten Reihenfolge als Zahlen dargestellt.
$$w, b \in \{0...9\}$$
Weiß $w$ und Schwarz $b$ müssen zu Beginn des Spiels alle ihre Steine legen. Weiß $w$ beginnt hierbei, daraus folgt
$$ (w = b) \lor (w+1 = b)$$

Das Spielbrett wird durch eine Tripel beschrieben, die die Ringe des Spielbretts beinhaltet. Alle zusammengehörigen Spielbrettpositionen, die sich auf dem gleichen Ring befinden, sind in der folgenden Abbilung durch eine Farbe markiert. Der äußere Ring ist $rot$, der mittlere $grün$ und der innere $blau$ dargestellt, damit gilt für einen Index $r$ der einen Ringe bezeichnet $r \in \{0...2\}$.

![](images/nmm-rings.png)

Die Spielpositionen (`cells`) $c$ auf den Ringen sind beginnend mit $c=0$ ab der oberen linken Ecke im Uhrzeigersinn durch nummeriert. Dadurch ergibt sich, dass $c \in \{0...7\}$. In der Abbilung sind die Koordinaten der Spielpositionen eingezeichnet, so liegt beispielsweise $\langle r,c \rangle = \langle 2,4 \rangle$ auf dem inneren Ring in der unteren rechten Ecke. An diesen Spielpositionen wird der Spieler $p$ oder eine leere Spielposition gespeichert, für diese gilt $p \in Player \cup \{'\ '\}$

Damit folgt die formale Definition
$$
\langle \langle w, b \rangle, \langle \langle p_{r, c} | \forall c \in \{0...7\} \rangle | \forall r \in \{0..2\} \rangle \rangle
$$
und der Startzustande $s_0$:

In [ ]:
s0 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))

## Folgezustände
Für die Definition des Spieles Mühle $G_{Nine Men's Morris}$ wird die Funktion `nextStates` benötigt. Diese nimmt einen Zustand `s` und einen Spieler `p` entgegen und errechnet mit diesen alle Folgezustände, die entstehen, wenn der gegebene Spieler einen legalen Zug spielt.

Da das Spiel in drei Phasen aufgeteilt ist, ist auch die Implementierung aus Gründen der Übersicht in die folgenden Phasen unterteilt:
1. Die erste Phase des Spiels heißt _placing_ Phase, in der die Spieler alle ihre Steine aus dem Stapel auf das Spielbrett legen müssen. Hierbei dürfen bereits Mühlen gelegt und geschlagen werden. Es muss ein Stein plaziert werden.
2. Die zweite Phase heißt _moving_ Phase und beginnt sobald der Stapel des Spielers leer ist. Beide Spieler kommen gleichzeitig in die zweite Phase in der die eigenen Steine nur noch entlang der Linien auf die nächste Spielposition geschoben werden dürfen. Es muss ein Stein bewegt werden.
3. Die letzte Phase heißt _flying_ Phase. Diese Phase beginnt für einen Spieler, sobald dieser nur noch $3$ Steine auf dem Spielbrett liegen hat und sein Stapel leer ist. Der Eintritt in die dritte Phase ist nicht vom gegnerischen Spieler abhängig, somit können Spieler für mehrere Züge alleine in der dritten Phase sein. Hierbei können die verbleibenden Steine nun von jeder Position aufgehoben werden und auf jede freie Position bewegt werden, ohne dass diese direkt nebeneinander liegen müssen. Es muss ein Stein bewegt werden.

In der ersten Phase, der _playing_ Phase, wird ein Stein vom Stapel genommen und auf ein freies Feld gelegt. Diese Züge werden in der Funktion `nextStatesPlace` implementiert, hierbei wird ein Spieler `p` und ein Zustand `s` erwartet, für angenommen wird, dass sich der aktuelle Zustand in der ersten Phase befindet.

1. Zunächst werden die Mühlen aus dem aktuellen Zustand mit Hilfe der Funktion `findMills` gespeichert.
2. Daraufhin wird auf jede freie Spielposition ein neuer Stein des Spielers `p` gelegt.
3. Nun werden unter Verwendung von dem Ergebnis aus _1.)_ und der Funktion `countNewMills` alle neu entstandenen Mühlen gezählt. Für diese Anzahl gilt $a \in \{0,1,2\}$.
4. Um für die neu entstandenen Mühlen die entsprechende Anzahl an Steinen zu schlagen, wird die Hilfsfunktion `poundMills` verwendet.
5. Zum Schluss wird vom Stapel des Spielers `p` ein Stein weg genommen und der Stapel mit den Spielbrettern wieder zu Zuständen vereint.

In [ ]:
def nextStatesPlace(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Place a stone in any empty cell
    placeBoards = {
        place(board, (r, c), p)
        for (r, c) in findEmptyCells(board)
    }
    # Calculate how many new mills were created
    boardMills = {
        board: countNewMills(board, mills, p)
        for board in placeBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    # Remove one stone from the players stache
    (cw, cb) = (cw-1, cb) if p == 'w' else (cw, cb-1)

    # Return all possible states
    return { ((cw, cb), board) for board in boards }

In der _moving_ Phase (2) wird ein Stein des aktuellen Spielers entlang der Linien zu einer benachbarten leeren Spielposition geschoben. Die Implementierung `nextStatesMove` wird wieder eine Zustand `s` und ein Spieler `p` erwartet, die sich in der zweiten Phase befinden.

In [ ]:
def nextStatesMove(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Choose any stone of the player and move it to an empty neighbor
    moveBoards = {
        move(board, src, des)
        for src in findCellsOf(board, p)
        for des in findNeighboringEmptyCells(board, src)
    }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in moveBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    return { ((cw, cb), board) for board in boards }

Die letzte Hilfsfunktion `nextStatesFly` wird in der Phase 3 verwendet und erwartet äquivalent zu den anderen Hilfsmethoden einen Zustand `s` und Spieler `p` in der Phase 3. Hier wird ein Stein des Spieler `p` an eine andere Position auf dem Spielbrett bewegt. Diese Position muss keine Nachbarposition sein und Steine können übersprungen werden.

In [ ]:
def nextStatesFly(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Choose any stone of the player and move it to an empty neighbor
    moveBoards = {
        move(board, src, des)
        for src in findCellsOf(board, p)
        for des in findEmptyCells(board)
    }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in moveBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    return { ((cw, cb), board) for board in boards }

Die Implementierung der `nextStates` Funktion führt nun die vorher definierten Funktionen zusammen. Die aktuelle Phase für den gegebenen Zustand `s` und Spieler `p` wird mit der Hilfsfunktion `playerPhase` errechnet und auf Grund dessen eine Fallunterscheidung ausgeführt, sodass gilt

$$
nextStates(s, p) = \begin{cases}
nextStatesPlace(s, p) & falls & playerPhase(s, p) = 1 \\
nextStatesMove(s, p) & falls & playerPhase(s, p) = 2 \\
nextStatesFly(s, p) & falls & playerPhase(s, p) = 3
\end{cases}
$$

In [ ]:
def nextStates(s, p):
    phase = playerPhase(s, p)
    if phase == 1:
        return nextStatesPlace(s, p)
    elif phase == 2:
        return nextStatesMove(s, p)
    else:
        return nextStatesFly(s, p)

## Game Progess

In [ ]:
def finished(s, p, ns=None):
    if not ns:
        ns = nextStates(s, p);
    return not hasEnoughStones(s, 'w') or \
           not hasEnoughStones(s, 'b') or \
           len(ns) == 0

Die Funktion `utility(s, p)` nimmt als Parameter einen Zustand `s`, für den gilt `finished(s) = true`, und einen Spieler `p`. Das Resultat ist ein Element der Menge `{-1, 0, 1}` und ist wie folgt zu interpretieren:
 * `-1`: Der Spieler `p` hat bei gegebenem Zustand `s` verloren.
 * `0`: Weder Spieler `p` noch sein Gegner haben gewonnen, es handlet sich um ein Unentschieden.
 * `1`: Der Spieler `p` hat bei gegebenem Zustand `s` gewonnen.

In [ ]:
def utility(s, p):
    utility  = -1 if not hasEnoughStones(s, p) else 0
    utility +=  1 if not hasEnoughStones(s, opponent(p)) else 0
    if utility != 0:
        return utility
    utility  = -1 if len(nextStates(s, p)) == 0 else 0
    utility +=  1 if len(nextStates(s, opponent(p))) == 0 else 0
    return utility